In [1]:
from neuralprophet.forecaster_additional_models import NBeatsNP, LSTM, DeepAR
import pandas as pd
import numpy as np
import os

In [2]:
data_loc = '../../LIBRA/'
datasets = os.listdir(data_loc)

datasets_economics = [dataset for dataset in datasets if 'economics' in dataset]
datasets_finance = [dataset for dataset in datasets if 'finance' in dataset]
datasets_human = [dataset for dataset in datasets if 'human' in dataset]
datasets_nature = [dataset for dataset in datasets if 'nature' in dataset]



In [3]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

def smape(y_true, y_pred):
    return 200*np.mean(np.abs((y_true-y_pred))/(np.abs(y_true+y_pred)))
def mase(y_true, y_pred, y_pred_naive):
    
    naive_mae = mae(y_true, y_pred_naive)
    y_pred_mae = mae(y_true, y_pred)

    return y_pred_mae/naive_mae

def mues(y_true, y_pred):
    return(np.mean(np.maximum(np.sign(y_true-y_pred),0)))

def moes(y_true, y_pred):
    return(np.mean(np.maximum(np.sign(y_pred-y_true),0)))

def muas(y_true, y_pred):
    m = mues(y_true, y_pred)
    if m == 0:
        return 0
    else:
        return (1/m) * np.mean(np.maximum(y_true - y_pred, 0)/np.abs(y_true))

def moas(y_true, y_pred):
    m = moes(y_true, y_pred)
    if m == 0:
        return 0
    else:
        return (1/m) * np.mean(np.maximum(y_pred - y_true, 0)/np.abs(y_true))



In [4]:
usecase = 'economics'

datasets = {}

if usecase == 'economics':
    for dataset in datasets_economics:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'finance':
    for dataset in datasets_finance:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'human':
    for dataset in datasets_human:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'nature':
    for dataset in datasets_nature:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
    
    


In [5]:
datasets

{'economics_80.csv':               0
 0    118.546043
 1    104.810656
 2     86.660324
 3    100.003271
 4     95.392105
 ..          ...
 926  110.599140
 927  152.393959
 928  124.138307
 929  101.671139
 930   94.411006
 
 [931 rows x 1 columns],
 'economics_94.csv':              0
 0    16.855022
 1    16.855022
 2    16.907295
 3    16.959569
 4    17.011842
 ..         ...
 117  17.203512
 118  17.255786
 119  17.273210
 120  17.273210
 121  17.325484
 
 [122 rows x 1 columns],
 'economics_43.csv':                  0
 0    112607.239182
 1    104416.948874
 2    122801.816481
 3    108323.609959
 4    134909.726251
 ..             ...
 415  349798.002672
 416  326024.901529
 417  346496.243949
 418  332499.110138
 419  329682.807476
 
 [420 rows x 1 columns],
 'economics_57.csv':              0
 0    14.018281
 1    14.036616
 2    14.009113
 3    13.999945
 4    13.963274
 ..         ...
 139  13.899100
 140  13.963274
 141  13.954106
 142  13.963274
 143  13.889932
 
 [144 row

In [6]:
frequencies = pd.read_csv(data_loc + 'freq.csv')

In [7]:
frequencies[[col for col in frequencies.columns if usecase in col][0]].unique()

array([  1,   4,  12,  52,  24,   7,  91, 364])

In [8]:
mapping_frequencies_economics = {
    1 : 'D',
    4: 'Q',
    12: 'M',
    52: 'W',
    24: 'H',
    7: 'W',
    91: 'D',
    364: 'D', 
    360: 'D',
    168: 'D',
    672: 'D',
    96: 'H', 
    288: 'W',
    28: 'D',
    6: 'D',
    30: 'D',
    720: 'D'
    
}

In [9]:
for dataset, df in datasets.items():
    idx_ts = int(dataset.split('.')[0].split('_')[-1])-1
    freq = mapping_frequencies_economics[frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]]
    df_ts = df.copy(deep = True)
    len_ts = df_ts.shape[0]
    df_ts['ds'] = pd.date_range(start = '2000-01-01', periods = len_ts, freq = freq)
    print(freq)
    print(df_ts)
    df_ts.columns = ['y', 'ds']
    datasets.update({dataset: df_ts})
    df_ts.to_csv(f'../../LIBRA_updated/{dataset}', index = False)

D
              0         ds
0    118.546043 2000-01-01
1    104.810656 2000-01-02
2     86.660324 2000-01-03
3    100.003271 2000-01-04
4     95.392105 2000-01-05
..          ...        ...
926  110.599140 2002-07-15
927  152.393959 2002-07-16
928  124.138307 2002-07-17
929  101.671139 2002-07-18
930   94.411006 2002-07-19

[931 rows x 2 columns]
M
             0         ds
0    16.855022 2000-01-31
1    16.855022 2000-02-29
2    16.907295 2000-03-31
3    16.959569 2000-04-30
4    17.011842 2000-05-31
..         ...        ...
117  17.203512 2009-10-31
118  17.255786 2009-11-30
119  17.273210 2009-12-31
120  17.273210 2010-01-31
121  17.325484 2010-02-28

[122 rows x 2 columns]
M
                 0         ds
0    112607.239182 2000-01-31
1    104416.948874 2000-02-29
2    122801.816481 2000-03-31
3    108323.609959 2000-04-30
4    134909.726251 2000-05-31
..             ...        ...
415  349798.002672 2034-08-31
416  326024.901529 2034-09-30
417  346496.243949 2034-10-31
418  33249

D
              0         ds
0    621.632005 2000-01-01
1    596.963737 2000-01-02
2    606.881907 2000-01-03
3    605.356034 2000-01-04
4    666.645237 2000-01-05
..          ...        ...
926  800.667685 2002-07-15
927  809.822919 2002-07-16
928  976.142996 2002-07-17
929  743.956099 2002-07-18
930  721.068015 2002-07-19

[931 rows x 2 columns]
M
             0         ds
0    11.751942 2000-01-31
1    11.751942 2000-02-29
2    12.001824 2000-03-31
3    12.251706 2000-04-30
4    12.501588 2000-05-31
..         ...        ...
117  15.999936 2009-10-31
118  17.624169 2009-11-30
119  15.125349 2009-12-31
120  13.376175 2010-01-31
121  13.501116 2010-02-28

[122 rows x 2 columns]
W
               0         ds
0    1819.789013 2000-01-02
1    1598.750739 2000-01-09
2    1709.931667 2000-01-16
3    1635.811048 2000-01-23
4    1584.191332 2000-01-30
..           ...        ...
857  2371.722905 2016-06-05
858  2289.660791 2016-06-12
859  2522.611307 2016-06-19
860  1765.522131 2016-06-26
86

M
               0         ds
0    2953.266287 2000-01-31
1    2934.621280 2000-02-29
2    3027.846315 2000-03-31
3    3102.426343 2000-04-30
4    3177.006371 2000-05-31
..           ...        ...
319  5339.827183 2026-08-31
320  5227.957141 2026-09-30
321  5358.472190 2026-10-31
322  5339.827183 2026-11-30
323  5283.892162 2026-12-31

[324 rows x 2 columns]
D
             0         ds
0   157.760815 2000-01-01
1   136.613960 2000-01-02
2   149.792435 2000-01-03
3   168.027766 2000-01-04
4   132.783008 2000-01-05
..         ...        ...
81  301.957843 2000-03-22
82  348.388979 2000-03-23
83  347.929265 2000-03-24
84  406.466210 2000-03-25
85  387.004974 2000-03-26

[86 rows x 2 columns]
M
              0         ds
0    338.402916 2000-01-31
1    333.974710 2000-02-29
2    334.528236 2000-03-31
3    341.032162 2000-04-30
4    344.076554 2000-05-31
..          ...        ...
173  552.894110 2014-06-30
174  552.063821 2014-07-31
175  556.215264 2014-08-31
176  556.492027 2014-09-30
17

W
             0         ds
0    69.352955 2000-01-02
1    66.104898 2000-01-09
2    55.366421 2000-01-16
3    61.265954 2000-01-23
4    57.686462 2000-01-30
..         ...        ...
926  43.302206 2017-10-01
927  50.328617 2017-10-08
928  47.080559 2017-10-15
929  35.878074 2017-10-22
930  12.213653 2017-10-29

[931 rows x 2 columns]
M
             0         ds
0    12.832260 2000-01-31
1    12.730142 2000-02-29
2    12.253596 2000-03-31
3    11.862147 2000-04-30
4    11.725991 2000-05-31
..         ...        ...
139  11.487717 2011-08-31
140  12.287635 2011-09-30
141  12.100420 2011-10-31
142  11.555795 2011-11-30
143   9.462394 2011-12-31

[144 rows x 2 columns]
Q
            0         ds
0   38.590147 2000-03-31
1   29.391620 2000-06-30
2   27.626482 2000-09-30
3   29.990757 2000-12-31
4   38.830844 2001-03-31
..        ...        ...
67  33.078223 2016-12-31
68  40.655590 2017-03-31
69  31.641436 2017-06-30
70  30.239208 2017-09-30
71  33.922785 2017-12-31

[72 rows x 2 columns]

In [10]:
df_ts

,y,ds
0,18242.822666,2000-01-31
1,20165.686460,2000-02-29
2,24118.574272,2000-03-31
3,21339.632734,2000-04-30
4,21714.091494,2000-05-31
...,...,...
171,31712.501604,2014-04-30
172,28649.404865,2014-05-31
173,33188.663791,2014-06-30
174,35730.407979,2014-07-31


In [14]:
datasets

{'economics_80.csv':               y         ds
 0    118.546043 2000-01-01
 1    104.810656 2000-01-02
 2     86.660324 2000-01-03
 3    100.003271 2000-01-04
 4     95.392105 2000-01-05
 ..          ...        ...
 926  110.599140 2002-07-15
 927  152.393959 2002-07-16
 928  124.138307 2002-07-17
 929  101.671139 2002-07-18
 930   94.411006 2002-07-19
 
 [931 rows x 2 columns],
 'economics_94.csv':              y         ds
 0    16.855022 2000-01-31
 1    16.855022 2000-02-29
 2    16.907295 2000-03-31
 3    16.959569 2000-04-30
 4    17.011842 2000-05-31
 ..         ...        ...
 117  17.203512 2009-10-31
 118  17.255786 2009-11-30
 119  17.273210 2009-12-31
 120  17.273210 2010-01-31
 121  17.325484 2010-02-28
 
 [122 rows x 2 columns],
 'economics_43.csv':                  y         ds
 0    112607.239182 2000-01-31
 1    104416.948874 2000-02-29
 2    122801.816481 2000-03-31
 3    108323.609959 2000-04-30
 4    134909.726251 2000-05-31
 ..             ...        ...
 415  349

In [12]:
valid_p = 0.2

In [13]:
from neuralprophet.utils.df_utils import split_df

metrics = {}

counter = 0
for dataset_name, df in datasets.items():
#     if counter == 1:
#         break
    
    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]
    freq = mapping_frequencies_economics[frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]]
    
    n_lags = freq_number
    n_forecasts = 3
    
    m = LSTM(n_lags = n_lags,
            n_forecasts=n_forecasts,
            num_hidden_layers=1,
            d_hidden=64,
            learning_rate=0.1,
            epochs=50,
            batch_size=None,
            loss_func="Huber",
            optimizer="AdamW",
            train_speed=None,
            normalize="auto",
            impute_missing=True,
            lstm_bias = True,
            lstm_bidirectional = False)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        'smape': smapes,
        'mase': mases,
        'mues': mueses,
        'moes': moeses,
        'muas': muases,
        'moas': moases
        
    }})
    
    counter += 1

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 110 K 
1 | linear    | Linear       | 195   
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
110 K     Trainable params
0         Non-trainable params
110 K     Total params
0.441     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16

  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 20.0 K
1 | linear    | Linear       | 195   
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
20.2 K    Trainable params
0         Non-trainable params
20.2 K    Total params
0.081     Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16














  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 20.0 K
1 | linear    | Linear       | 195   
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
20.2 K    Trainable params
0         Non-trainable params
20.2 K    Total params
0.081     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

In [19]:
pd.DataFrame(metrics).to_csv('lstm_libra.csv', index = False)

In [16]:
metrics

{'economics_80.csv': {'smape': 14.227959482889146,
  'mase': 0.6793891006101452,
  'mues': 0.6845238095238094,
  'moes': 0.3154761904761904,
  'muas': 0.10425659722191864,
  'moas': 0.2198020913237889},
 'economics_94.csv': {'smape': 0.7190360801962151,
  'mase': 3.1163183418894147,
  'mues': 0.36507936507936506,
  'moes': 0.6349206349206349,
  'muas': 0.001503754856503776,
  'moas': 0.006099359626964676},
 'economics_43.csv': {'smape': 5.829709637480135,
  'mase': 1.0738051638414605,
  'mues': 0.725,
  'moes': 0.275,
  'muas': 0.05530059414642341,
  'moas': 0.028149931598778904},
 'economics_57.csv': {'smape': 0.1298987500371631,
  'mase': 0.4951778316399436,
  'mues': 0.1733333333333333,
  'moes': 0.8266666666666667,
  'muas': 0.0003500760304027347,
  'moas': 0.0014781010942533108},
 'economics_56.csv': {'smape': 3.0017473773169367,
  'mase': 2.069085805064148,
  'mues': 0.4533333333333333,
  'moes': 0.5466666666666666,
  'muas': 0.018881944477294157,
  'moas': 0.020638514320898648},

In [231]:
np.array(y_rolled).shape

(112, 3)

In [234]:
np.array(y_predicted).shape

(112, 3)

In [209]:
datasets.items()

dict_items([('economics_80.csv',               y         ds
0    118.546043 2000-01-01
1    104.810656 2000-01-02
2     86.660324 2000-01-03
3    100.003271 2000-01-04
4     95.392105 2000-01-05
..          ...        ...
926  110.599140 2002-07-15
927  152.393959 2002-07-16
928  124.138307 2002-07-17
929  101.671139 2002-07-18
930   94.411006 2002-07-19

[931 rows x 2 columns]), ('economics_94.csv',              y         ds
0    16.855022 2000-01-31
1    16.855022 2000-02-29
2    16.907295 2000-03-31
3    16.959569 2000-04-30
4    17.011842 2000-05-31
..         ...        ...
117  17.203512 2009-10-31
118  17.255786 2009-11-30
119  17.273210 2009-12-31
120  17.273210 2010-01-31
121  17.325484 2010-02-28

[122 rows x 2 columns]), ('economics_43.csv',                  y         ds
0    112607.239182 2000-01-31
1    104416.948874 2000-02-29
2    122801.816481 2000-03-31
3    108323.609959 2000-04-30
4    134909.726251 2000-05-31
..             ...        ...
415  349798.002672 2034-08-

In [207]:
idx_ts

98

In [186]:
n_lags

12

In [187]:
freq_number

12

In [188]:
m.make_future_dataframe(vl, n_historic_predictions = True)

,ds,y,t,y_scaled
0,2002-01-31,72.434388,1.044286,0.475743
1,2002-02-28,75.966344,1.084286,0.503912
2,2002-03-31,125.119406,1.128571,0.895931
3,2002-04-30,113.051888,1.171429,0.799687
4,2002-05-31,57.914122,1.215714,0.359937
5,2002-06-30,120.312021,1.258571,0.857590
6,2002-07-31,NaN,1.302857,NaN


In [189]:
vl

,y,ds
0,86.071665,2002-01-06
1,87.445203,2002-01-07
2,83.717027,2002-01-08
3,96.961864,2002-01-09
4,60.366870,2002-01-10
...,...,...
190,110.599140,2002-07-15
191,152.393959,2002-07-16
192,124.138307,2002-07-17
193,101.671139,2002-07-18


In [190]:
df

,y,ds
0,118.546043,2000-01-01
1,104.810656,2000-01-02
2,86.660324,2000-01-03
3,100.003271,2000-01-04
4,95.392105,2000-01-05
...,...,...
926,110.599140,2002-07-15
927,152.393959,2002-07-16
928,124.138307,2002-07-17
929,101.671139,2002-07-18


In [191]:
freq

'M'

In [147]:
len(df)

931

In [148]:
len(tr)

748

In [151]:
len(vl) - freq_number

183